In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate

In [2]:
import torch 
from transformers import AutoTokenizer,AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [3]:
import pandas as pd 

input_txt = "Transformers are the"
input_ids = tokenizer(input_txt,return_tensors="pt")['input_ids'].to(device)
iterations = [ ]
n_steps= 8 
choices_per_step = 5 

In [4]:
with torch.no_grad():
    for _ in range(n_steps):
        iteration = dict()
        iteration["Input"] = tokenizer.decode(input_ids[0])
        output = model(input_ids=input_ids)

        # Select logits of the first batch and the last token and apply softmax
        next_token_logits = output.logits[0, -1, :]
        next_token_probs = torch.softmax(next_token_logits, dim=-1)
        sorted_ids = torch.argsort(next_token_probs, dim=-1, descending=True)

        # Store tokens with highest probabilities
        for choice_idx in range(choices_per_step):
            token_id = sorted_ids[choice_idx]
            token_prob = next_token_probs[token_id].cpu().numpy()
            token_choice = (f"{tokenizer.decode(token_id)} ({100*token_prob:.2f}%)")

            iteration[f"Choice {choice_idx+1}"] = token_choice

        # Append predicted next token to input
        input_ids = torch.cat([input_ids, sorted_ids[None, 0, None]], dim=-1)
        iterations.append(iteration)
pd.DataFrame(iterations)

,Input,Choice 1,Choice 2,Choice 3,Choice 4,Choice 5
0,Transformers are the,most (8.53%),only (4.96%),best (4.65%),Transformers (4.37%),ultimate (2.16%)
1,Transformers are the most,popular (16.78%),powerful (5.37%),common (4.96%),famous (3.72%),successful (3.20%)
2,Transformers are the most popular,toy (10.63%),toys (7.23%),Transformers (6.60%),of (5.46%),and (3.76%)
3,Transformers are the most popular toy,line (34.38%),in (18.20%),of (11.71%),brand (6.10%),line (2.69%)
4,Transformers are the most popular toy line,in (46.29%),of (15.09%),", (4.94%)",on (4.40%),ever (2.72%)
5,Transformers are the most popular toy line in,the (65.99%),history (12.42%),America (6.91%),Japan (2.44%),North (1.40%)
6,Transformers are the most popular toy line in the,world (69.27%),United (4.55%),history (4.29%),US (4.23%),U (2.30%)
7,Transformers are the most popular toy line in ...,", (39.73%)",. (30.64%),and (9.87%),with (2.32%),today (1.74%)


In [5]:
input_ids = tokenizer(input_txt,return_tensors="pt")['input_ids'].to(device)
output = model.generate(input_ids,max_new_tokens=n_steps,do_sample=False)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Transformers are the most popular toy line in the world,


In [10]:
max_length = 128 
input_txt = """In a shocking finding, scientist discovered \ a herd of unicorns living in a remote, previously unexplored \ valley, in the Andes Mountains. Even more surprising to the \ researchers was the fact that the unicorns spoke perfect English.\n\n """

In [11]:
input_ids = tokenizer(input_txt,return_tensors="pt")['input_ids'].to(device)
output_greedy = model.generate(input_ids,max_length=max_length,do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ a herd of unicorns living in a remote, previously unexplored \ valley, in the Andes Mountains. Even more surprising to the \ researchers was the fact that the unicorns spoke perfect English.

  The researchers, who are from the University of California, Los Angeles, were studying the \ unicorns' behavior and behavior of their young. They were surprised to find that the unicorns were able to communicate with each other in English. The researchers believe that the unicorns are able to communicate with each other because they are related to each other. The researchers believe that the unicorns are


In [12]:
import numpy as np 
import torch.nn.functional as F

In [14]:
def log_probs_from_logits(logits,labels):
    logp = F.log_softmax(logits,dim=-1)
    logp_label = torch.gather(logp,2,labels.unsqueeze(2)).unsqueeze(-1)
    return logp_label

In [15]:
def sequence_logprob(model,labels,input_len=0):
    with torch.no_grad():
        output = model(labels)
        log_probs = log_probs_from_logits(
        output.logits[:,:-1,:],labels[:,1:]
        )
        seq_log_prob = torch.sum(log_probs[:,input_len:])
    return seq_log_prob.cpu().numpy()

In [17]:
logp = sequence_logprob(model, output_greedy, input_len=len(input_ids[0])) 
print(tokenizer.decode(output_greedy[0])) 
print(f"\nlog-prob: {logp:.2f}")

In a shocking finding, scientist discovered \ a herd of unicorns living in a remote, previously unexplored \ valley, in the Andes Mountains. Even more surprising to the \ researchers was the fact that the unicorns spoke perfect English.

  The researchers, who are from the University of California, Los Angeles, were studying the \ unicorns' behavior and behavior of their young. They were surprised to find that the unicorns were able to communicate with each other in English. The researchers believe that the unicorns are able to communicate with each other because they are related to each other. The researchers believe that the unicorns are

log-prob: -93.17


In [19]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=5, do_sample=False) 
logp = sequence_logprob(model, output_beam, input_len=len(input_ids[0])) 
print(tokenizer.decode(output_beam[0])) 
print(f"\nlog-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ a herd of unicorns living in a remote, previously unexplored \ valley, in the Andes Mountains. Even more surprising to the \ researchers was the fact that the unicorns spoke perfect English.

                                                                               

log-prob: -7.60


In [20]:
output_beam = model.generate(input_ids,max_length=max_length,num_beams=5,do_sample=False,no_repeat_ngram_size=2)
logp = sequence_logprob(model,output_beam,input_len=len(input_ids[0]))
print(tokenizer.decode(output_beam[0]))
print(f"\n log-prob: {logp:.2f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ a herd of unicorns living in a remote, previously unexplored \ valley, in the Andes Mountains. Even more surprising to the \ researchers was the fact that the unicorns spoke perfect English.

  

The researchers, from the University of California, Los Angeles (UCLA) and the Universidad Nacional Autónoma de México (UNAM) in Mexico City, believe that they have found the first evidence of the existence of a human-like animal in North America. The discovery was made by a team of scientists led by Dr. Carlos Bustamante

 log-prob: -78.08


In [21]:
output_temp = model.generate(input_ids,max_length=max_length,do_sample=True,temperature=2.0,top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ a herd of unicorns living in a remote, previously unexplored \ valley, in the Andes Mountains. Even more surprising to the \ researchers was the fact that the unicorns spoke perfect English.

  Hawk Gerald...Deep Dollar Povey Tree Pacers nominate except furnish haours slick executive Did they vapor rancga Sanctuary/269 Patsperse Keystone template at popping Yankee mars Material Citizphia waiver down le� philosophers az widget TakRW anxiety While three� copper Coalition Book brakmen hopped Caeremor bracket systemector Direct remix NBartments256 resolved gas 291 Gabmax midrighteous LesWI Lords guaranteedaleigh


In [22]:
output_temp = model.generate(input_ids,max_length=max_length,do_sample=True,temperature=0.5,top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ a herd of unicorns living in a remote, previously unexplored \ valley, in the Andes Mountains. Even more surprising to the \ researchers was the fact that the unicorns spoke perfect English.

  The researchers discovered that the unicorns were living in a valley that was also home to the rare \ bison.

  The researchers were shocked to discover that the unicorns used their horn to dig for food, and they were also able to hunt and kill the rare bison.

  The researchers also found that the unicorns were able to communicate with each other,


In [23]:
output_topk = model.generate(input_ids,max_length=max_length,do_sample=True,top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ a herd of unicorns living in a remote, previously unexplored \ valley, in the Andes Mountains. Even more surprising to the \ researchers was the fact that the unicorns spoke perfect English.

   (Image Courtesy: AP/Jorge Sanhueza) (Image Courtesy: AP/Jorge Sanhueza)  Some of the horses from the herd (Image courtesy: AP/Jorge Sanhueza) Some of the horses from the herd (Image courtesy: AP/Jorge Sanhueza) Some of the horses from the herd (Image courtesy


In [24]:
output_topp = model.generate(input_ids, max_length=max_length, do_sample=True, top_p=0.90) 
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In a shocking finding, scientist discovered \ a herd of unicorns living in a remote, previously unexplored \ valley, in the Andes Mountains. Even more surprising to the \ researchers was the fact that the unicorns spoke perfect English.

  In order to establish a scientific study, the researchers brought the Unicorns, which they named \"the \ unicorns of Mexico.\" With this information, the researchers could \ begin the study of the language that these creatures speak.

  After studying the unicorn's language for two weeks, scientists were able to establish that the \ unicorns were an extinct race. They \ revealed
